# Big Project – Session 3: Chatbot & Sentiment
Create a friendly Game Master persona and test sentiment reactions. Saves outputs into `game_project/dialogue/`.

In [1]:
# Setup
!pip -q install transformers accelerate torch --extra-index-url https://download.pytorch.org/whl/cu121
import os, json, textwrap
DIALOGUE_DIR = "/mnt/data/nicegpu_ai_workshop/game_project/dialogue"
os.makedirs(DIALOGUE_DIR, exist_ok=True)
print('Saving prompts to:', DIALOGUE_DIR)

Saving prompts to: /mnt/data/nicegpu_ai_workshop/game_project/dialogue


In [2]:
from transformers import pipeline
device = 0 if __import__('torch').cuda.is_available() else -1
sentiment = pipeline('sentiment-analysis', device=device)
gen = pipeline('text-generation', model='gpt2', device=device)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0
Device set to use cuda:0


### Define your Game Master persona

In [3]:
persona = "You are a friendly Game Master. Encourage players, give short hints, and keep answers under 40 words. Be school-appropriate."
with open(f"{DIALOGUE_DIR}/persona.txt","w") as f: f.write(persona)
print('Saved persona to dialogue/persona.txt')

Saved persona to dialogue/persona.txt


### Test the chatbot and sentiment

In [4]:
def game_master_reply(persona, context, max_new_tokens=60):
    prompt = f"""{persona}
Context: {context}
Reply:"""
    out = gen(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.9)[0]['generated_text']
    return out

samples = [
    "Player guessed cat and got it right!",
    "Player guessed car but the truth was dog.",
]
for s in samples:
    fb = sentiment(f"I guessed: {s}")[0]
    reply = game_master_reply(persona, f"{s} Sentiment={fb['label']}")
    print("\nContext:", s, "| Sentiment:", fb)
    print("GM:", reply)

# Save some canned lines to reuse in the game loop if desired
canned = {
    "correct": ["Nice! You nailed it!", "Spot on!"],
    "incorrect": ["Close! Try again.", "Not quite—look carefully at the details."],
}
import json, os
with open(f"{DIALOGUE_DIR}/canned_lines.json","w") as f:
    json.dump(canned, f, indent=2)
print("Saved canned_lines.json")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Context: Player guessed cat and got it right! | Sentiment: {'label': 'POSITIVE', 'score': 0.9994888305664062}
GM: You are a friendly Game Master. Encourage players, give short hints, and keep answers under 40 words. Be school-appropriate.
Context: Player guessed cat and got it right! Sentiment=POSITIVE
Reply:
A good start for you and your guild is to encourage people to ask, get in touch, and even post your questions to the thread.
Players are encouraged to ask questions on the boards or on the forums as an example of how to get them to engage. If you are being generous,

Context: Player guessed car but the truth was dog. | Sentiment: {'label': 'NEGATIVE', 'score': 0.9907868504524231}
GM: You are a friendly Game Master. Encourage players, give short hints, and keep answers under 40 words. Be school-appropriate.
Context: Player guessed car but the truth was dog. Sentiment=NEGATIVE
Reply: You have the right to suggest a particular set of rules and/or objectives. Make it your goal to mee